<a href="https://colab.research.google.com/github/Sep-eg/kaggle_survey2021/blob/main/kaggle_survey_2021_GHK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir ~/.kaggle # api키 보관용 폴더 생성
!cp kaggle.json ~/.kaggle # api키 파일을 보관용 폴더에 복사
!kaggle competitions download -c kaggle-survey-2021 # kaggle survey 2021 데이터 다운로드

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
# from pywaffle import Waffle
warnings.filterwarnings('ignore')

plt.style.use('seaborn') # 생서할 그래프의 스타일시트를 변경 ggplot, grayscale, seaborn 등 다양한 파라미터 사용가능
sns.set(font_scale=2.5) # 앞으로 생성할 모든 그래프의 폰트 사이즈를 지정

In [ ]:
response = pd.read_csv("/content/kaggle_survey_2021_responses.csv.zip")

In [ ]:
response = response.loc[:][1:] # 질문을 담고있는 첫번째 행 제거
response.reset_index(drop=True, inplace=True) # 인덱스 초기화

In [ ]:
def total_answer(res, column):
    """
    컬럼별로 나누어진 응답을 통합하는 함수
    column : 컬럼명
    """

    # 해당 컬럼명이 들어간 컬럼 추출
    col = res.columns[res.columns.str.contains(column)]
    tmp = res[col]
    # 결측치 ''로 채우기
    tmp = tmp.fillna('')
    # 컬럼별 응답 세미콜론(;)으로 더하기
    total = tmp.apply(';'.join, axis=1) # ','로 구분할 경우 나중에 응답횟수를 셀때 잘못나뉘는 경우가 있어 ';'사용

    # 정규표현식으로 필요없는 세미콜론(;) 처리
    import re
    ls = []
    for i in total:
        ls.append(re.sub(';{2,}', ';', i).strip(';')) ## 세미콜론이 2개 이상인 경우 하나로 바꾸기

    # 나눠진 응답 삭제
    res.drop(col, axis=1, inplace=True)

    # 통합 응답 추가
    res[column] = ls

In [ ]:
def job_group(res):
    res['Q5'] = res['Q5'].replace(['Business Analyst', 'Data Analyst',
                                   'Product Manager', 'Program/Project Manager',
                                   'Statistician'],'DA')
    
    res['Q5'] = res['Q5'].replace(['Data Engineer', 'DBA/Database Engineer',
                                   'Machine Learning Engineer', 'Software Engineer',
                                   'Developer Relations/Advocacy'], 'DE')
    
    res['Q5'] = res['Q5'].replace(['Data Scientist', 'Research Scientist'], 'DS')
    
    res['Q5'] = res['Q5'].replace(['Student', 'Currently not employed', 'Other'], 'OTHER')

In [ ]:
# Apply total_answer() to only multiple-type questions

q_list_supplementary = [27,29,30,31,32,34,36,37,38] # supplementary question list

for i in range(7, 43):
    if i in q_list_supplementary:
        num_A = 'Q' + str(i) + '_A'
        num_B = 'Q' + str(i) + '_B'
        total_answer(response ,num_A)
        total_answer(response ,num_B)

    else:
        num = 'Q'+str(i)
        total_answer(response ,num)

cond = ((response['Q27_B'] != '') | (response['Q29_B'] != '') | (response['Q30_B'] != '') | 
        (response['Q31_B'] != '') | (response['Q32_B'] != '') | (response['Q34_B'] != '') | 
        (response['Q36_B'] != '') | (response['Q37_B'] != '') | (response['Q38_B'] != ''))

response['professional'] = 'Professional'
response.loc[cond, 'professional'] = 'Non-professional' 

In [ ]:
job_group(response)

##<질문 예시>

Q. 직군별 성별 비율 
Q2 Gender Q5 JobTitle

Q. 연령에 따른 성별 분포 
Q1 Age Q5 JobTitle

Q. 프로그래밍 공부 최소 어느 정도 해야 취직이 가능할까요? 직군별 프로그래밍 코드 사용기간(최저) 
Q5 JobTitle Q6 YearsToCode

Q. 직군별 평균 나이 
Q1 Age Q5 JobTitle

Q. 직군별 최종학력 
Q4 FormalEducation Q5 JobTitle

Q. 직군별로 어떤 언어를 사용하시나요? 
Q5 JobTitle Q7 Language_regular

Q. 직군별로 어떤 언어를 추천하시나요? 
Q5 JobTitle Q8 Language_first

Q. 직군별로 평소에 사용하는 시각화 라이브러리는 무엇인가요? 
Q5 JobTitle Q14 Vis_library

Q. 직군별로 사용하는 ML 알고리즘은 무엇인가요? 
Q5 JobTitle Q17 ML_algorithm

Q. 직군별로 현재 종사하는 산업의 비율?은 어떠한가요? 
Q5 JobTitle Q20 JobIndustry

Q. 직군별 회사 규모에 따른 연봉은 어떠한가요? 
Q5 JobTitle Q21 CompanySize 회사규모 Q25 Year_Compensation 연봉 ($USD)

Q. 데이터분석 또는 머신러닝 앱을 어디에 배포하나요? 
Q5 JobTitle Q39 PublicToDeploy

Q. 데이터사이언스를 배울때 어떤 플랫폼을 사용했나요? 
Q5 JobTitle Q40 PlatformToLear_DS

Q. 데이터사이언스와 관련된 주제를 다루는 미디어 중 가장 선호하는 채널은 무엇인가요? 
Q5 JobTitle Q42 MediaSourceToLearn

### 질문 1. 직군별 성별의 비율

In [ ]:
job_gender = pd.crosstab(response['Q5'], response['Q2'])

In [ ]:
job_gender.plot.bar(stacked=True, figsize=(15,9));

###질문 2. 연령에 따른 성별의 분포

In [ ]:
age_gender = pd.crosstab(response['Q1'], response['Q2'])

In [ ]:
age_gender.plot.bar(stacked=True, figsize=(15,9));

### 질문 3. 프로그래밍 공부기간과 취직

In [ ]:
job_study = pd.crosstab(response['Q5'], response['Q6'])

In [ ]:
job_study.plot.bar(stacked=True, figsize=(15,9));

이부분은 직군별 비교로 살펴보는것도 재밌지만, 실제로 프로와 non프로를 비교해 볼 수 있을 것 같습니다.

In [ ]:
pro_study = pd.crosstab(response['Q6'], response['professional'])

In [ ]:
pro_study.plot.bar(stacked=True, figsize=(15,9));

확실히 코드를 사용한 기간이 길수록 프로의 비율이 커지는것으로 보입니다. 이를 조금 바꿔서 직군별로 프로들은 어느정도 기간동안 코드를 사용해 왔는지 살펴보겠습니다.

In [ ]:
pro_job_study = pd.crosstab(response[response['professional'] == 'Professional']['Q5'],
                            response[response['professional'] == 'Professional']['Q6'])

In [ ]:
pro_job_study.plot.bar(stacked=True, figsize=(15,9));

프로들의 데이터만 남겨서 살펴보았을때 DS, DE직군에 비해 DA직군이 코드를 1년미만으로 사용한 사람의 비율이 조금 큰 것을 확인할 수 있습니다.

###질문 4. 직군별 나이 분포

In [ ]:
job_age = pd.crosstab(response['Q1'], response['Q5'])

In [ ]:
job_age

In [ ]:
plt.figure(figsize=(18,9))
sns.lineplot(data = job_age)

학생 그룹이 포함된 Other을 제외한다면 대체로 모든 직군에서 유사한 분포를 그리고 있으나, DE그룹은 조금 더 연령대가 낮은것으로 보입니다. 유의미한 차이일지는 모르겠네요.

###질문 5. 직군별 최종학력

In [ ]:
job_edu = pd.crosstab(response['Q4'], response['Q5'])

In [ ]:
plt.figure(figsize=(18,9))
g = sns.lineplot(data = job_edu)
g.set_xticklabels(['Bd', 'Dd', 'NotAsw', 'Md','HS','PD','WOBd']);

DE, DA직군은 학, 석사학위자의 비중이 높은반면, DS직군은 석사학위를 보유한 인원의 비율은 유사하지만, 학사학위자의 비율이 낮아지고 박사학위자가 크게 많아진 것을 볼 수 있습니다.

###질문 6. 직군별 사용 언어

In [ ]:
# 2차원 딕셔너리를 만들기위해 답변 리스트 생성
language_list = ['Python', 'R', 'SQL', 'C', 'C++', 'Java',
                 'Javascript', 'Julia', 'Swift', 'Bash',
                 'MATLAB', 'None', 'Other']

In [ ]:
# {직군: {언어: 응답횟수, 언어: 응답횟수, ...}} , {직군:{}...}, ... 형태로 이루어진 2차원 딕셔너리 생성
language_dict = {job : {lang : 0 for lang in language_list} for job in response['Q5'].unique()}

for row in response.iterrows():
    if len(row[1]['Q7']) > 0: # 답변 길이가 0이 아니면('' 걸러내는 용도)
        for lang in row[1]['Q7'].split(';'): # 답변을 ;을 기준으로 나눔
            language_dict[row[1]['Q5']][lang] += 1 # 직군별 사용언어에 응답횟수를 더한다.

In [ ]:
pd.DataFrame(language_dict).plot.bar(figsize=(15,6));

어느정도 차이가 눈에 보이기는 하지만 Other의 비율이 커 쉽게 알아보기가 힘듭니다. Other을 제외한 DE,DS,DA직군만 보겠습니다. 

In [ ]:
pd.DataFrame(language_dict).loc[:, ['DA','DE','DS']].plot.bar(stacked=True)

파이썬은 가장 많이 사용되는 언어이며, 세 직군 모두 사용 비율이 유사해보입니다. 하지만 통계에 특화된 것으로 알려진 R같은 경우 DE직군에서는 거의 사용되지 않고, C, C++, Java 등 개발에 특화된 것으로 알려진 언어는 DA, DS직군에 비해 DE직군의 사용률이 높은것으로 보입니다.

### 질문 7. 직군별 추천 언어

In [ ]:
re_language_dict = {job : {lang : 0 for lang in language_list} for job in response['Q5'].unique()}

for row in response.iterrows():
    if len(row[1]['Q8']) > 0:
        for lang in row[1]['Q8'].split(';'):
            re_language_dict[row[1]['Q5']][lang] += 1

In [ ]:
pd.DataFrame(re_language_dict).plot.bar(stacked=True, figsize=(15,6));

거의 대부분의 추천언어가 Python에 몰려있는 것을 볼 수있습니다. 다른 언어들과 차이가 커 확실히 알아보기 어렵지만, 사용언어와 마찬가지로 Other을 제외한 뒤 살펴보겠습니다.

In [ ]:
f, ax = plt.subplots(1,2, figsize=(20,9))
pd.DataFrame(language_dict).loc[:, ['DA','DE','DS']].plot.bar(stacked=True, ax=ax[0])
ax[0].set_title('Use language')
pd.DataFrame(re_language_dict).loc[:, ['DA','DE','DS']].plot.bar(stacked=True, ax=ax[1])
ax[1].set_title('Recommend language')
plt.show()

실제 사용언어와 추천언어의 비교를 위해 두 그래프를 함께 표시하였습니다. Python의 추천비중이 압도적으로 많지만, R, SQL과 같이 함께 추천받은 언어들은 실제 사용하는 언어와 유사한 분포를 나타내는 것으로 보입니다.

### 질문 8. 직군별 시각화 라이브러리

In [ ]:
visual_list = ['Matplotlib', 'Seaborn', 'Plotly / Plotly Express',
               'Ggplot / ggplot2', 'Shiny', 'D3 js', 'Altair', 'Bokeh',
               'Geoplotlib', 'Leaflet / Folium', 'None', 'Other']

In [ ]:
visual_dict = {job : {visual : 0 for visual in visual_list} for job in response['Q5'].unique()}

for row in response.iterrows():
    if len(row[1]['Q14']) > 0:
        for visual in row[1]['Q14'].split(';'):
            visual_dict[row[1]['Q5']][visual.strip()] += 1

In [ ]:
pd.DataFrame(visual_dict).plot.bar(figsize=(15,6));

### 질문 9. 직군별로 사용하는 ML 알고리즘

In [ ]:
ML_list = ['Linear or Logistic Regression', 'Decision Trees or Random Forests',
           'Gradient Boosting Machines (xgboost, lightgbm, etc)', 'Bayesian Approaches',
           'Evolutionary Approaches', 'Dense Neural Networks (MLPs, etc)',
           'Convolutional Neural Networks', 'Generative Adversarial Networks',
           'Recurrent Neural Networks', 'Transformer Networks (BERT, gpt-3, etc)',
           'None', 'Other']

In [ ]:
ML_dict = {job : {ML : 0 for ML in ML_list} for job in response['Q5'].unique()}

for row in response.iterrows():
    if len(row[1]['Q17']) > 0:
        for ML in row[1]['Q17'].split(';'):
            ML_dict[row[1]['Q5']][ML] += 1

In [ ]:
g = pd.DataFrame(ML_dict).plot.bar(figsize=(15,6));
g.set_xticklabels(['Linear or LR', 'DT or RF', 'GBM', 'Bayesian',
                   'Evolutionay', 'DNN', 'CNN', 'GAN', 'RNN', 'Transformer',
                   'None', 'Other']);

DS직군이 확실히 다양한 종류의 ML알고리즘을 사용하고 있음을 확인할 수 있고, DA직군은 DE 혹은 DS직군에 비해 딥러닝 분야의 알고리즘의 활용도가 낮은것을 볼 수 있습니다.

### 질문 10. 직군별 종사 산업

In [ ]:
response['Q5'].unique()

In [ ]:
response['Q20'].unique()

In [ ]:
Q5_Q20_notnan = response[response['Q20']!=''][['Q5','Q20']]

In [ ]:
job_industry = pd.crosstab(Q5_Q20_notnan['Q20'], Q5_Q20_notnan['Q5'])

In [ ]:
g = job_industry.plot.bar(figsize=(18, 9))
g.set_xticklabels(['Edu','Finance','Broadcating','Tech','Energy',
                   'Gover/PS','Enter','Insurance','Manufacturing',
                   'Marketing','Medical','Security','Service',
                   'OnlineBiz','OnlineServ','Other','Retail',
                   'Transport']) # xticklabel이 너무 길어서 축약
plt.show()

In [ ]:
g = job_industry.loc[:,['DA','DE','DS']].plot.bar(figsize=(18,9))
g.set_xticklabels(['Edu','Finance','Broadcating','Tech','Energy',
                   'Gover/PS','Enter','Insurance','Manufacturing',
                   'Marketing','Medical','Security','service',
                   'OnlineBiz','OnlineServ','Other','Retail',
                   'Transport'])
plt.show()

각 직군별로 종사 산업은 대체로 비슷한 수준의 규모를 보이지만, DS같은경우 Academics/Education, DE는 Computers/Technology 분야에 특히 많이 종사하고 있는것으로 나타났습니다.

###질문 11. 직군별 회사 규모에 따른 연봉







In [ ]:
# 각 직군별로 회사 규모, 연봉이 결측치가 아닌값만 선별
DA_income = response[(response['Q5'] == 'DA') & (response['Q21'] != '') &
                     (response['Q25'] != '')][['Q21','Q25']] 

DS_income = response[(response['Q5'] == 'DS') & (response['Q21'] != '') &
                     (response['Q25'] != '')][['Q21','Q25']]

DE_income = response[(response['Q5'] == 'DE') & (response['Q21'] != '') &
                     (response['Q25'] != '')][['Q21','Q25']]

OTHER_income = response[(response['Q5'] == 'OTHER') & (response['Q21'] != '') &
                        (response['Q25'] != '')][['Q21','Q25']]

In [ ]:
# 응답데이터가 오브젝트형일 경우 정렬 및 시각화에 어려움이 있어 변환
match_income = {'$0-999':1, '1,000-1,999':2, '2,000-2,999':3, '3,000-3,999':4,
                '4,000-4,999':5, '5,000-7,499':6, '7,500-9,999':7, '10,000-14,999':8,
                '15,000-19,999':9, '20,000-24,999':10, '25,000-29,999':11, '30,000-39,999':12,
                '40,000-49,999':13, '50,000-59,999':14, '60,000-69,999':15, '70,000-79,999':16,
                '80,000-89,999':17, '90,000-99,999':18, '100,000-124,999':19, '125,000-149,999':20,
                '150,000-199,999':21, '200,000-249,999':22, '250,000-299,999':23, '300,000-499,999':24,
                '$500,000-999,999':25, '>$1,000,000':26}

In [ ]:
plt.figure(figsize=(18,15))
g = sns.violinplot(x = DA_income['Q21'], y = DA_income['Q25'].replace(match_income),
                order = ['0-49 employees', '50-249 employees', '250-999 employees',
                        '1000-9,999 employees', '10,000 or more employees']) # x축 순서가 랜덤하게 섞이므로 보기편하게 정렬
g.set_xticklabels(['0-49', '50-249', '250-999', '1000-9,999', '10,000 or more'])
g.set_yticks(range(0,len(match_income),2)) # y축값을 모두 표시할경우 너무 빽뺵하여 2칸마다 표시
g.set_yticklabels(list(match_income.keys())[::2]) # 시각화 자료를 알아보기 쉽도록 원래 표기로 변환
plt.show()

In [ ]:
plt.figure(figsize=(18,15))
g = sns.violinplot(x = DS_income['Q21'], y = DS_income['Q25'].replace(match_income),
                order = ['0-49 employees', '50-249 employees', '250-999 employees',
                        '1000-9,999 employees', '10,000 or more employees'])
g.set_xticklabels(['0-49', '50-249', '250-999', '1000-9,999', '10,000 or more'])
g.set_yticks(range(0,len(match_income),2))
g.set_yticklabels(list(match_income.keys())[::2])
plt.show()

In [ ]:
plt.figure(figsize=(18,15))
g = sns.violinplot(x = DE_income['Q21'], y = DE_income['Q25'].replace(match_income),
                order = ['0-49 employees', '50-249 employees', '250-999 employees',
                        '1000-9,999 employees', '10,000 or more employees'])
g.set_xticklabels(['0-49', '50-249', '250-999', '1000-9,999', '10,000 or more'])
g.set_yticks(range(0,len(match_income),2))
g.set_yticklabels(list(match_income.keys())[::2])
plt.show()

In [ ]:
plt.figure(figsize=(18,15))
g = sns.violinplot(x = OTHER_income['Q21'], y = OTHER_income['Q25'].replace(match_income),
                order = ['0-49 employees', '50-249 employees', '250-999 employees',
                        '1000-9,999 employees', '10,000 or more employees'])
g.set_xticklabels(['0-49', '50-249', '250-999', '1000-9,999', '10,000 or more'])
g.set_yticks(range(0,len(match_income),2))
g.set_yticklabels(list(match_income.keys())[::2])
plt.show()

대부분의 직업군에서 회사의 규모가 커질수록 높은 수입을 가진 고용인층이 두터워지는 모습을 보입니다.

###질문 12. 데이터분석 또는 머신러닝 앱 배포처

In [ ]:
Deploy_list = ['Plotly Dash', 'Streamlit', 'NBViewer', 'GitHub',
               'Personal blog', 'Kaggle', 'Colab', 'Shiny',
               'I do not share my work publicly', 'Other']

In [ ]:
Deploy_dict = {job : {Deploy : 0 for Deploy in Deploy_list} for job in response['Q5'].unique()}

for row in response.iterrows():
    if len(row[1]['Q39']) > 0:
        for Deploy in row[1]['Q39'].split(';'):
            Deploy_dict[row[1]['Q5']][Deploy.strip()] += 1

In [ ]:
pd.DataFrame(Deploy_dict).plot.bar(figsize=(15,6));

###질문 13. 데이터사이언스를 배운 플랫폼

In [ ]:
LearnPl_list = ['Coursera', 'edX', 'Kaggle Learn Courses', 'DataCamp',
               'Fast.ai', 'Udacity', 'Udemy', 'LinkedIn Learning',
               'Cloud-certification programs (direct from AWS, Azure, GCP, or similar)',
               'University Courses (resulting in a university degree)', 'None', 'Other']

In [ ]:
LearnPl_dict = {job : {LearnPl : 0 for LearnPl in LearnPl_list} for job in response['Q5'].unique()}

for row in response.iterrows():
    if len(row[1]['Q40']) > 0:
        for LearnPl in row[1]['Q40'].split(';'):
            LearnPl_dict[row[1]['Q5']][LearnPl.strip()] += 1

In [ ]:
g = pd.DataFrame(LearnPl_dict).plot.bar(figsize=(15,6));
g.set_xticklabels(['Coursera', 'edX', 'Kaggle', 'DataCamp',
               'Fast.ai', 'Udacity', 'Udemy', 'LinkedIn',
               'Cloud programs', 'University Courses', 'None', 'Other'])
plt.show()

### 질문 14. 데이터사이언스 관련 선호 미디어

In [ ]:
Media_list = ['Twitter (data science influencers)', 'Email newsletters (Data Elixir, O\'Reilly Data & AI, etc)',
              'Reddit (r/machinelearning, etc)', 'Kaggle (notebooks, forums, etc)', 
              'Course Forums (forums.fast.ai, Coursera forums, etc)',
              'YouTube (Kaggle YouTube, Cloud AI Adventures, etc)',
              'Podcasts (Chai Time Data Science, O’Reilly Data Show, etc)',
              'Blogs (Towards Data Science, Analytics Vidhya, etc)',
              'Journal Publications (peer-reviewed journals, conference proceedings, etc)',
              'Slack Communities (ods.ai, kagglenoobs, etc)', 'None', 'Other']

In [ ]:
Media_dict = {job : {Media : 0 for Media in Media_list} for job in response['Q5'].unique()}

for row in response.iterrows():
    if len(row[1]['Q42']) > 0:
        for Media in row[1]['Q42'].split(';'):
            Media_dict[row[1]['Q5']][Media.strip()] += 1

In [ ]:
g = pd.DataFrame(Media_dict).plot.bar(figsize=(15,6))
g.set_xticklabels(['Twitter', 'Email newsletters', 'Reddit', 'Kaggle',
                   'Course Forums', 'YouTube', 'Podcasts', 'Blogs',
                   'Journal Publications', 'Slack Communities', 'None', 'Other'])
plt.show()

## Pro vs non-Pro

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot(response['professional'])
plt.title('Professional vs Non-professional')
plt.xlabel('')
plt.show()

이번 2021 kaggle survey에서 자신을 Professional 이라고 소개한 사람과, Non-professional이라고 소개한 사람의 수는 거의 유사합니다. 이렇듯 캐글은 자신의 실력을 증명하고 싶은 전문가 집단과, 실력을 키우고 배움을 얻고 싶은 비전문가 집단 모두가 활발히 이용하는 서비스입니다. 그렇다면 전문가 집단이 사용하는 기술과 전문가가 되고 싶은 집단이 익숙해지고 싶은 기술은 어떤 차이, 혹은 공통점을 보일까요?

### 1. Cloud Computing Platforms & Product
ReportLinker는 클라우드 컴퓨팅 시장이 2021년 4,453억 달러에서 2026년 9,473억 달러 규모로 성장할 것으로 전망한 보고서[[Link]](https://www.reportlinker.com/p05749258/Cloud-Computing-Market-by-Service-Deployment-Model-Organization-Size-Workload-Vertical-And-Region-Global-Forecast-to.html?utm_source=GNW)를 발표했습니다.